## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from pyvi import ViTokenizer
import re
from tqdm.notebook import tqdm
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
import torch
from torch.utils.data import DataLoader
from datasets import Dataset
import faiss
import time

## Preprocessing

In [ ]:
corpus = pd.read_csv('corpus.csv')
public_test = pd.read_csv('public_test.csv')
train = pd.read_csv('train.csv')


In [ ]:
# Remove rows with NaN values and reset index of corpus dataframe
corpus = corpus.dropna().reset_index(drop=True)

# Remove rows with NaN values and reset index of public_test dataframe
public_test = public_test.dropna().reset_index(drop=True)

# Remove rows with NaN values and reset index of train dataframe
train = train.dropna().reset_index(drop=True)

In [ ]:
corpus['text'] = corpus['text'].apply(lambda x: x.replace('\n', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.replace('\t', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.replace('\r', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.replace('  ', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.lower())

# Đường dẫn đến hai file .txt
file1_path = 'vietnamese-stopwords-dash.txt'  # Thay đổi đường dẫn tới file thứ nhất
file2_path = 'vietnamese-stopwords.txt'  # Thay đổi đường dẫn tới file thứ hai
def read_stopwords(file_path):
    stopwords = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stopword = line.strip()  # Tách theo khoảng trắng
            stopwords.append(stopword)  # Thêm stopword vào list
    return stopwords
stopwords1 = read_stopwords(file1_path)
stopwords2 = read_stopwords(file2_path)

vietnamese_stopwords = set(stopwords1 + stopwords2)

def remove_punctuation(text):
    no_punct_text = re.sub(r'[^\w\s]', '', text)  # This will keep Vietnamese letters and spaces
    return no_punct_text

corpus['text'] = corpus['text'].apply(lambda x: ViTokenizer.tokenize(x))

corpus['text'] = corpus['text'].apply(lambda x: remove_punctuation(x))

corpus['text'] = corpus['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))

corpus['text'] = corpus['text'].apply(lambda x: unidecode.unidecode(x))
public_test['question'] = public_test['question'].apply(lambda x: x.replace('\n', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.replace('\t', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.replace('\r', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.replace('  ', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.lower())

# Đường dẫn đến hai file .txt
file1_path = 'vietnamese-stopwords-dash.txt'  # Thay đổi đường dẫn tới file thứ nhất
file2_path = 'vietnamese-stopwords.txt'  # Thay đổi đường dẫn tới file thứ hai
def read_stopwords(file_path):
    stopwords = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stopword = line.strip()  # Tách theo khoảng trắng
            stopwords.append(stopword)  # Thêm stopword vào list
    return stopwords
stopwords1 = read_stopwords(file1_path)
stopwords2 = read_stopwords(file2_path)

vietnamese_stopwords = set(stopwords1 + stopwords2)

def remove_punctuation(text):
    no_punct_text = re.sub(r'[^\w\s]', '', text)  # This will keep Vietnamese letters and spaces
    return no_punct_text

public_test['question'] = public_test['question'].apply(lambda x: ViTokenizer.tokenize(x))

public_test['question'] = public_test['question'].apply(lambda x: remove_punctuation(x))

public_test['question'] = public_test['question'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))

public_test['question'] = public_test['question'].apply(lambda x: unidecode.unidecode(x))
train['question'] = train['question'].apply(lambda x: x.replace('\n', ' '))
train['question'] = train['question'].apply(lambda x: x.replace('\t', ' '))
train['question'] = train['question'].apply(lambda x: x.replace('\r', ' '))
train['question'] = train['question'].apply(lambda x: x.replace('  ', ' '))
train['question'] = train['question'].apply(lambda x: x.lower())

# Đường dẫn đến hai file .txt
file1_path = 'vietnamese-stopwords-dash.txt'  # Thay đổi đường dẫn tới file thứ nhất
file2_path = 'vietnamese-stopwords.txt'  # Thay đổi đường dẫn tới file thứ hai
def read_stopwords(file_path):
    stopwords = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stopword = line.strip()  # Tách theo khoảng trắng
            stopwords.append(stopword)  # Thêm stopword vào list
    return stopwords
stopwords1 = read_stopwords(file1_path)
stopwords2 = read_stopwords(file2_path)

vietnamese_stopwords = set(stopwords1 + stopwords2)

def remove_punctuation(text):
    no_punct_text = re.sub(r'[^\w\s]', '', text)  # This will keep Vietnamese letters and spaces
    return no_punct_text

train['question'] = train['question'].apply(lambda x: ViTokenizer.tokenize(x))

train['question'] = train['question'].apply(lambda x: remove_punctuation(x))

train['question'] = train['question'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))

train['question'] = train['question'].apply(lambda x: unidecode.unidecode(x))
train['context'] = train['context'].apply(lambda x: x.replace('\n', ' '))
train['context'] = train['context'].apply(lambda x: x.replace('\t', ' '))
train['context'] = train['context'].apply(lambda x: x.replace('\r', ' '))
train['context'] = train['context'].apply(lambda x: x.replace('  ', ' '))
train['context'] = train['context'].apply(lambda x: x.lower())

# Đường dẫn đến hai file .txt
file1_path = 'vietnamese-stopwords-dash.txt'  # Thay đổi đường dẫn tới file thứ nhất
file2_path = 'vietnamese-stopwords.txt'  # Thay đổi đường dẫn tới file thứ hai
def read_stopwords(file_path):
    stopwords = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stopword = line.strip()  # Tách theo khoảng trắng
            stopwords.append(stopword)  # Thêm stopword vào list
    return stopwords
stopwords1 = read_stopwords(file1_path)
stopwords2 = read_stopwords(file2_path)

vietnamese_stopwords = set(stopwords1 + stopwords2)

def remove_punctuation(text):
    no_punct_text = re.sub(r'[^\w\s]', '', text)  # This will keep Vietnamese letters and spaces
    return no_punct_text

train['context'] = train['context'].apply(lambda x: ViTokenizer.tokenize(x))

train['context'] = train['context'].apply(lambda x: remove_punctuation(x))

train['context'] = train['context'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))

train['context'] = train['context'].apply(lambda x: unidecode.unidecode(x))

In [ ]:
train.to_csv('train.csv', index=False)
public_test.to_csv('public_test.csv', index=False)
corpus.to_csv('corpus.csv', index=False)

## Training

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from transformers import RagRetriever, RagTokenForGeneration, DPRContextEncoder, DPRQuestionEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer
import faiss

# Check if MPS is available
device = torch.device("mps" if torch.mps.is_available() else "cpu")

# Step 1: Load your dataset
corpus_df = pd.read_csv('corpus.csv')  # Assuming columns: 'context' and 'cid'
train_df = pd.read_csv('train.csv')  # Assuming columns: 'question', 'context', 'cid', 'qid'
public_test_df = pd.read_csv('public_test.csv')  # Assuming columns: 'question', 'qid'

# Step 2: Extract passages and their IDs
passages = corpus_df['text'].tolist()
passage_ids = corpus_df['cid'].tolist()

# Step 3: Load DPR context and question encoders with appropriate tokenizers
dpr_context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base", ignore_mismatched_sizes=True).to(device)
dpr_context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
dpr_question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)
dpr_question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

# Step 4: Embed all passages using DPRContextEncoder
def embed_passages(passages, batch_size=32):
    all_embeddings = []
    
    # Wrap the for loop with tqdm to display the progress
    for i in tqdm(range(0, len(passages), batch_size), desc="Embedding Passages"):
        batch = passages[i:i + batch_size]
        inputs = dpr_context_tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
        
        # Move inputs to the MPS
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # Disable gradient computation for faster inference
        with torch.no_grad():
            embeddings = dpr_context_encoder(**inputs).pooler_output
        
        all_embeddings.append(embeddings)
    
    # Concatenate all the embeddings into a single tensor
    return torch.cat(all_embeddings, dim=0)

# Get embeddings for all passages
passage_embeddings = embed_passages(passages)

# Step 5: Use FAISS to index the passage embeddings
d = passage_embeddings.shape[1]  # Dimensionality of the embeddings

# Create a FAISS index for MPS
res = faiss.StandardGpuResources()  # Create MPS resources
index = faiss.IndexFlatIP(d)  # Inner Product (dot product) index on CPU
mps_index = faiss.index_cpu_to_gpu(res, 0, index)  # Move the index to MPS
mps_index.add(passage_embeddings.cpu().numpy())  # Add embeddings to the index (move to CPU for FAISS)

# Step 6: Create a custom retriever using FAISS
class FaissRetriever(RagRetriever):
    def __init__(self, index, passages, tokenizer, top_k=5):
        self.index = index
        self.passages = passages
        self.tokenizer = tokenizer
        self.top_k = top_k

    def retrieve(self, question_input_ids, question_hidden_states, question_attention_mask=None):
        # Tokenize and embed the question using DPRQuestionEncoder
        question_embeds = self._embed_question(question_input_ids)
        
        # Search FAISS index for top-k relevant documents
        _, indices = self.index.search(question_embeds.cpu().numpy(), self.top_k)  # Move to CPU for FAISS
        
        # Return top-k passages
        retrieved_passages = [self.passages[i] for i in indices[0]]
        return retrieved_passages

    def _embed_question(self, question_input_ids):
        question = self.tokenizer.batch_decode(question_input_ids, skip_special_tokens=True)
        inputs = dpr_question_tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=128)
        
        # Move inputs to the MPS
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        with torch.no_grad():
            question_embeddings = dpr_question_encoder(**inputs).pooler_output
        return question_embeddings

# Step 7: Initialize the custom retriever and the RAG model
retriever = FaissRetriever(index=mps_index, passages=passages, tokenizer=dpr_question_tokenizer, top_k=10)
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever).to(device)

# Step 8: Predict top-k documents for each query in public_test.csv with progress bar
top_k_predictions = []

# Use tqdm to display progress while generating predictions
for idx, row in tqdm(public_test_df.iterrows(), total=len(public_test_df), desc="Processing Queries"):
    question = row['question']
    qid = row['qid']
    
    # Tokenize question
    input_ids = dpr_question_tokenizer(question, return_tensors="pt", max_length=128, truncation=True, padding=True).input_ids
    
    # Move inputs to the MPS
    input_ids = input_ids.to(device)
    
    # Generate top-k passage predictions
    retrieved_passages = retriever.retrieve(input_ids, None)
    
    # Retrieve the passage IDs (cid) for the predicted passages
    retrieved_cids = [corpus_df.loc[corpus_df['context'] == passage]['cid'].values[0] for passage in retrieved_passages]
    
    # Append result (qid followed by top-k cids)
    top_k_predictions.append(f"{qid} " + " ".join(map(str, retrieved_cids)))

# Step 9: Save predictions to predict.txt
with open('predict.txt', 'w') as f:
    for prediction in top_k_predictions:
        f.write(prediction + '\n')

print("Predictions saved to predict.txt.")
